In [1]:
!pip install razdel

In [2]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML
import re
from collections import Counter
import nltk
from sklearn.model_selection import train_test_split

In [3]:
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста. Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).

Подсказки:
- нужно будет добавить еще один тэг <start>
- можете использовать тот же подход с матрицей вероятностей, но по строкам хронить биграмы, а по колонкам униграммы - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так) - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным

Создадим корпус, токенизируем, поделим на train (для сбора статистик для марковской модели) и val (для подсчета переплексии).

In [4]:
!wget https://github.com/mannefedov/compling_nlp_hse_course/raw/refs/heads/master/notebooks/lm_intro/2ch_corpus.txt

--2025-06-02 07:54:26--  https://github.com/mannefedov/compling_nlp_hse_course/raw/refs/heads/master/notebooks/lm_intro/2ch_corpus.txt
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/refs/heads/master/notebooks/lm_intro/2ch_corpus.txt [following]
--2025-06-02 07:54:26--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/refs/heads/master/notebooks/lm_intro/2ch_corpus.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20632515 (20M) [text/plain]
Saving to: ‘2ch_corpus.txt’

2ch_corpus.txt      100%[===================>]  19.68M  --.-KB/s    i

In [48]:
dvach = open("/content/2ch_corpus.txt").read()

In [49]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [50]:
from nltk.tokenize import sent_tokenize
dvach = [
    ["<start>"] * 2 + normalize(text) + ["<end>"]
    for text in sent_tokenize(dvach[:5000000])
]
dvach_train, dvach_val = train_test_split(
    dvach , test_size=50, random_state=42
)

In [51]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0, len(tokens)-n+1):
        ngrams.append(" ".join(tokens[i:i+n]))
    return ngrams

In [52]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()

for sentence in dvach_train:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence))
    trigrams_dvach.update(ngrammer(sentence, n=3))

In [53]:
from scipy.sparse import lil_matrix, csr_matrix, csc_matrix

In [54]:
matrix_dvach = lil_matrix(
    (len(bigrams_dvach), len(unigrams_dvach))
)

id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}

id2bigram_dvach = list(bigrams_dvach)
bigram2id_dvach = {bigram:i for i, bigram in enumerate(id2bigram_dvach)}

for trigram in trigrams_dvach:
    bigram, word = trigram.rsplit(" ", 1)
    matrix_dvach[
        bigram2id_dvach[bigram],
        word2id_dvach[word]
    ] = (trigrams_dvach[trigram] / bigrams_dvach[bigram])

matrix_dvach = csc_matrix(matrix_dvach)

In [55]:
def generate(
    matrix, id2word, ngram2id, total_tokens=100, n=1
):
    start = " ".join(["<start>"] * n)
    text = []
    current_idx = ngram2id[start]
    prev_words = ["<start>"] * (n - 1)

    for i in range(total_tokens):
        chosen_idx = np.random.choice(
            matrix.shape[1],
            p=matrix[current_idx].toarray()[0]
        )
        chosen_word = id2word[chosen_idx]
        text.append(chosen_word)

        if chosen_word == "<end>":
            prev_words = ["<start>"] * (n - 1)
            chosen_word = "<start>"

        prev_words.append(chosen_word)
        current_idx = ngram2id[" ".join(prev_words)]
        prev_words = prev_words[1:]

    generation = re.sub(r"<end> ?", "\n", " ".join(text))
    return generation

In [56]:
for i in range(5):
    print(f"Генерация {i+1}:")
    print(generate(matrix_dvach, id2word_dvach, bigram2id_dvach, n=2))
    print('==='*50)

Генерация 1:
даа даа прямо в штаны не проиграл когда дошел до третьего уровня и загрустил we re acting without your permission we feel that those things have to say anything else i think honestly 8 in the future i plan to take someone s life 
или мертвый 
сейчас свежачек залью 
ай-да отака азазааз 
до следующей серии 
я был бы пидаром дрочил бы дохуя всё больше напоминает российскую империю сменившую вывеску 
застрелиться 
а вот на боевом наву оно будет устроено 
жалко что только селедочка заебись а у всех синдром нинужно star citizen выйдет весь
Генерация 2:
наверно только на харкаче в разделе 
от смуты нет 
обсуждается тотальный контроль за тем как выставлять товар на продажу 
5 
хром не пускает на фишинговую страницу уведомляя об опасности 
да всем похуй чо там у банды реднеков что купили себе дом в лесу сычует радагаст второй карапузов-хоббиков ебет третий у саурона шестерит 
няшноу меня такое было 
уже лечу с завода 
взял бы лучше 
говорит что искусственная пизда не торчит и жопа 

In [57]:
def compute_join_proba_markov_assumption(
    text, smaller_ngram_counts, ngram_counts,
    n=1, tokenized=True
):
    prob = 0
    if tokenized:
        tokenized_text = text
    else:
        tokenized_text = ["<start>"] * n + normalize(text) + ["<end>"]

    for ngram in ngrammer(tokenized_text, n=n+1):
        left_ngram, word = ngram.rsplit(" ", 1)
        if left_ngram in smaller_ngram_counts and ngram in ngram_counts:
            prob += np.log(
                ngram_counts[ngram] / smaller_ngram_counts[left_ngram]
            )
        else:
            prob += np.log(2e-5)
    return prob, len(tokenized_text) - n - 1

In [58]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)

In [59]:
def mean_perplexity(
    data_list, smaller_ngram_counts,
    ngram_counts, n=1, tokenized=True,
    print_logs=True
):
    ppl_sum = 0
    for sent in data_list:
        if print_logs:
            if tokenized:
                print(" ".join(sent[n:-1]))
            else:
                print(sent)

        current_ppl = perplexity(
            *compute_join_proba_markov_assumption(
                sent, smaller_ngram_counts, ngram_counts,
                n=n, tokenized=tokenized
            )
        )
        ppl_sum += current_ppl
        if print_logs:
            print("Перплексия:", current_ppl, "\n")
    return ppl_sum / len(data_list)

In [60]:
perplexity_val = mean_perplexity(
    dvach_val, bigrams_dvach, trigrams_dvach, n=2
)

может как-то стек портится
Перплексия: 203730.95147289796 

ок
Перплексия: 17985.74999999999 

тогда все трапы куны
Перплексия: 3662.1163487889753 

для сборок он собрался c учить имбецил ебаный
Перплексия: 104278.32945614542 

нихуя такого нет работают на станках еще 40-50 х годов
Перплексия: 4382.621969118742 

спейсмариныдо превращения в маринов марины до превращения в маринов ну конечно блять
Перплексия: 4015.5747283092774 

не нужна нам твоя колбаса и джинсы будем кору жрать но добъемся того чтобы раисся по количеству яхт и поместий для депутатов порвала гребаных америкосов
Перплексия: 58814.53696473338 

если человек рожает ребёнка он должен понимать что это поступок не на 18 лет а на всю жизнь этого ребёнка
Перплексия: 17180.192574549692 

пиздец ты упоротый
Перплексия: 800.9772284137746 

покупаю только продукты на каждый день зависимость есть брат жив
Перплексия: 7419.969299362334 

но если есть символ который намерено вырезан на спинке если стул обит и покрашен то уже станови

In [62]:
print(perplexity_val)
train_perp=mean_perplexity(
    dvach_train[1000:2000], bigrams_dvach, trigrams_dvach, n=2,
    print_logs=False
)
print(f'перплексия на трейне: {train_perp}')

10711697.024497004
перплексия на трейне: 1945.0846621500932


Задание № 2* (2 балла).
Измените функцию generate_with_beam_search так, чтобы она работала с моделью, которая учитывает два предыдущих слова. Сравните получаемый результат с первым заданием. Также попробуйте начинать генерацию не с нуля (подавая <start> <start>), а с какого-то промпта. Но помните, что учитываться будут только два последних слова, так что не делайте длинные промпты.

In [63]:
# сделаем класс чтобы хранить каждый из лучей
class Beam:
    def __init__(self, sequence: list, score: float):
        self.sequence: list = sequence
        self.score: float = score

In [64]:
def generate_with_beam_search(
    matrix, id2word, ngram2id,
    total_tokens=100, max_beams=5,
    n=1, prompt=""
):
    # изначально у нас один луч с заданным началом (start по дефолту)
    initial_node = Beam(
        sequence=["<start>"] * n + normalize(prompt),
        score=np.log1p(0)
    )
    beams = [initial_node]

    for i in range(total_tokens):
        # делаем total_tokens шагов генерации
        new_beams = []
        # на каждом шаге продолжаем каждый из имеющихся лучей
        for beam in beams:
            # лучи которые уже закончены не продолжаем (но и не удаляем)
            if beam.sequence[-1] == "<end>":
                new_beams.append(beam)
                continue

            # наша языковая модель предсказывает на основе предыдущей n-граммы
            # достанем его из beam.sequence
            last_id = ngram2id[" ".join(beam.sequence[-n:])]

            # посмотрим вероятности продолжений для предыдущей n-граммы
            probas = matrix[last_id].toarray()[0]

            # возьмем топ самых вероятных продолжений
            top_idxs = probas.argsort()[:-(max_beams+1):-1]
            for top_id in top_idxs:
                # иногда вероятности будут нулевые, такое не добавляем
                if not probas[top_id]:
                    break

                # создадим новый луч на основе текущего и варианта продолжения
                new_sequence = beam.sequence + [id2word[top_id]]
                # скор каждого луча это произведение вероятностей (или сумма логарифмов)
                new_score = (beam.score + np.log1p(probas[top_id])) / len(new_sequence)
                new_beam = Beam(sequence=new_sequence, score=new_score)
                new_beams.append(new_beam)
        # отсортируем лучи по скору и возьмем только топ max_beams
        beams = sorted(new_beams, key=lambda x: x.score, reverse=True)[:max_beams]

    sorted_sequences = sorted(beams, key=lambda x: x.score, reverse=True)
    sorted_sequences = [" ".join(beam.sequence[n:-1]) for beam in sorted_sequences]
    return sorted_sequences

In [67]:
generate_with_beam_search(
    matrix_dvach, id2word_dvach,
    bigram2id_dvach, max_beams=10, n=2,
)

['а вот и всё такое',
 'да я и так далее',
 'а вот и всё такое но для всех остальных сейчас вобщето нет',
 'а вот и всё',
 'я не знаю',
 'а вот и всё такое но разница между assembly language и assembler для каждого вида или для каждого патрона r нажимают или удерживают',
 'ну и что',
 'но я не знаю',
 'а вот и всё такое но для всех остальных сейчас',
 'а вот и всё такое но разница между assembly language и assembler для каждого вида или для каждого патрона r нажимают запихивают сначала в магазин со своим говном']

In [71]:
# пример из генерации
# общество дебилов которое занимается тем же успехом можно задвигать бред про кислотный диагноз
generate_with_beam_search(
    matrix_dvach, id2word_dvach,
    bigram2id_dvach, max_beams=10, n=2,
    prompt="общество дебилов"
)

['общество дебилов которое занимается тем же содержимым',
 'общество дебилов которое занимается тем же браком брак или недокомплект',
 'общество дебилов которое занимается тем же канонам что и требовалось доказать',
 'общество дебилов которое занимается тем же геймплеем но разными квестами и боевыми системами',
 'общество дебилов которое занимается тем же самым',
 'общество дебилов которое занимается тем же браком',
 'общество дебилов которое занимается тем же канонам что и требовалось доказать чего-то конкретного не намечается',
 'общество дебилов которое занимается тем же самым уровнем качества и проработки их игры тоже говно',
 'общество дебилов которое занимается тем же самым уровнем качества и проработки их игры выглядят как ущербные инди-поделки',
 'общество дебилов которое занимается тем же самым уровнем качества и проработки их игры выглядят как говно']

In [72]:
generate_with_beam_search(
    matrix_dvach, id2word_dvach,
    bigram2id_dvach, max_beams=10, n=2,
    prompt="зачем общаться"
)
# зачем общаться с парнями на личных автомобилях

['зачем общаться с топовыми людьми',
 'зачем общаться с слабоумными идиотами',
 'зачем общаться с начальником он вроде натурал',
 'зачем общаться с девушками',
 'зачем общаться с парнем',
 'зачем общаться с десятком пёзд друзей стеснительность пройдёт',
 'зачем общаться с десятком мужиков за раз',
 'зачем общаться с начальником он вроде больше подходит жопу отопыриваешь прогибаешься вперед как можно было бы интересно',
 'зачем общаться с начальником он вроде больше подходит the brian но я не знаю',
 'зачем общаться с начальником он вроде больше подходит жопу отопыриваешь прогибаешься вперед как можно было бы интересно самостоятельно это всё равно не понимаю']

Вывод по бим серч:
варианты выглядят более логичными
как будто бы модель выбирает варианты в которых она более уверена(а мы тут еще температуру не прикручиваем)